### Import Package

In [ ]:
!pip install stockstats

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from stockstats import StockDataFrame

from sklearn.svm import SVC

from math import floor

import pandas_datareader.data as web
import pickle
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model = "SVC"

volatility_folder = "/content/drive/MyDrive/fyp/Final report/4. Backtesting/Volatility_result/"
backtest_summary_folder = volatility_folder + "backtest_summary/"
record_folder = volatility_folder + "trade_record/"
backtest_folder= "/content/drive/MyDrive/fyp/Final report/4. Backtesting/"
model_folder = backtest_folder+model+"/"

Mounted at /content/drive


# Backtesting Setting

In [ ]:
start_date = []
num_period = 10
period_year = 1

start = 2022-(num_period+period_year-1)
for i in range(num_period):
    start_date.append((start+i)*10000+101)


date_range_list={}
for i in start_date:
    date_range_list[i] = (i-101)+(period_year-1)*10000+1231

print(date_range_list)

stock_list = {"BlockChain":["COIN","NVDA","FB"],
              "Airline":["BA","GD","LMT",],
              "Traveling":["UBER","ABNB","MAR","BKNG"],
              "Semiconductors":["INTC","NVDA","QCOM","MU","AMD"],
              "Cloud Computing":["IBM", "AMZN","GOOG","CRM"],
              "Social Media":["TWTR","SNAP","PINS","FB"],
              "Entertainment":["DIS","NFLX","FB"],
              "Retail":["WMT","COST","TGT","BBY","HD"],
              "Franchise":["MCD", "YUM", "SBUX", "DPZ"],
              "Real Estate":["HST", "EQR", "AVB", "PLD", "SPG"],
              "Telecommunication":["T","TMUS","VZ","CMCSA","CHTR"],
              "Energy & Resources":["DOW", "DD"],
              "Luxury goods":["RACE","EL","PVH"]
                            }

start_date = "2000-01-01"
end_date = "2022-12-31"


do_nth_name = "Buy&Hold"
model_name = "WithModel"

backtest_col = ["Sector","Stock",
                "Range_Start","Range_End",
                do_nth_name+"_Profit",model_name+"_Profit",
                do_nth_name+"_CAGR(in %)",model_name+"_CAGR(in %)",
                "CAGR_Performance(in %)",
                do_nth_name+"_MDD(in %)",model_name+"_MDD(in %)",
                "MDD_Performance(in %)"]

{20120101: 20121231, 20130101: 20131231, 20140101: 20141231, 20150101: 20151231, 20160101: 20161231, 20170101: 20171231, 20180101: 20181231, 20190101: 20191231, 20200101: 20201231, 20210101: 20211231}


# Backtesting Module

In [ ]:
################### Input ##########################
# For hist_price_data: index=["date"], columns = ["Open"]
# For pred_action: index=["date"], columns = ["Action"] (Buy/Sell)
################### Output #########################
# 1. trading record
# 2. total profit
class backtest:
    hpd = ""
    pred_action=pd.DataFrame()
    trade_record=pd.DataFrame(index=[],
                              columns=["Action","Price","Position","Cash","Pos_Bal","Cash_Bal","Cum_Profit","Total_Bal"],
                             )
    capital = 0
    cash_balance = 0
    profit = 0
    handle_fee = 0
    position = 0
    last_price = 0
    do_nth_profit = 0
    num_year = 0
    _tested = False
    
    _stock_trough = 0
    _stock_peak = 0
    _stock_all_time_low = 0
    _stock_all_time_high = 0
    
    _portfolio_trough = 0
    _portfolio_peak = 0
    _portfolio_all_time_low = 0
    _portfolio_all_time_high = 0
    
    def __init__(self,hist_price_data,pred_action,capital,handling_fee,num_year=1):
        self.hpd = hist_price_data
        self.pred_action = pred_action
        self.capital = capital
        self.cash_balance = capital
        self._portfolio_trough = capital
        self._portfolio_peak = capital
        self._portfolio_all_time_low = capital
        self._portfolio_all_time_high = capital
        self.handle_fee = handling_fee
        self.num_year = num_year  
        
    def clear_trade_record(self,sec):
        self.trade_record=pd.DataFrame(index=[],
                                       columns=["Action","Price","Position","Cash","Pos_Bal","Cash_Bal","Cum_Profit","Total_Bal"],
                                       )
        print("Clearing trade_record...")
        time.sleep(sec)
        
    def start_test(self): 
        if not self._tested:
            status = "sell"
            self._tested = True
            print("Start Backtesting...")  
            self._stock_all_time_low = self.hpd.iloc[0,0]
            self._stock_all_time_high = self.hpd.iloc[0,0]
            self._stock_trough = self.hpd.iloc[0,0]
            self._stock_peak = self.hpd.iloc[0,0]
            # For loop to iterate the data
            for ind in self.pred_action.index:
                # Update latest price
                self.last_price = self.hpd.loc[ind,"Open"]
                
                # Mark All Time Low,High , Trough and Peak for MDD of stock price (using "Buy&Hold")
                if self.last_price > self._stock_all_time_high:
                    self._stock_all_time_high = self.last_price
                if self.last_price < self._stock_all_time_low:
                    self._stock_all_time_low = self.last_price
                if self.last_price > self._stock_all_time_low and self._stock_trough != self._stock_all_time_low:
                    self._stock_trough = self._stock_all_time_low
                if self.last_price < self._stock_all_time_high and self._stock_peak != self._stock_all_time_high:
                    self._stock_peak = self._stock_all_time_high
                
                # Do the action
                if self.pred_action.loc[ind,"Action"].lower() == "buy" and status == "sell":
                    self._buy(ind,self.last_price)
                    status = "buy"
                elif self.pred_action.loc[ind,"Action"].lower() == "sell" and status == "buy":
                    self._sell(ind,self.last_price)
                    status = "sell"
                else:
                    self._hold(ind,self.last_price) # newly added
                    
                
                curr_portfolio_val = self.get_capital()+self.get_profit()
                
                # Mark All Time Low,High , Trough and Peak for MDD of our portfolio balance (using our model)
                if curr_portfolio_val > self._portfolio_all_time_high:
                    self._portfolio_all_time_high = curr_portfolio_val
                if curr_portfolio_val < self._portfolio_all_time_low:
                    self._portfolio_all_time_low = curr_portfolio_val
                if curr_portfolio_val > self._portfolio_all_time_low and self._portfolio_trough != self._portfolio_all_time_low:
                    self._portfolio_trough = self._portfolio_all_time_low
                if curr_portfolio_val < self._portfolio_all_time_high and self._portfolio_peak != self._portfolio_all_time_high:
                    self._portfolio_peak = self._portfolio_all_time_high
                            
            # =================================================
            self._run_do_nothing() # Calculate do nothing profit
        else:
            print("Backtesting has been completed...")
              
                
        
    def _mark_down_record(self,date,action,price,pos_delta,cash_delta):
        self.trade_record.loc[date] = [action,price,pos_delta,cash_delta,
                                       round(self.position,4),round(self.cash_balance,3),
                                       round(self.get_profit(),3),round(self.get_amount(),3)]
        
    def _buy(self,date,price):
        # Assume use all money to buy all
        buy_pos = floor(self.cash_balance / price)
        for i in range(buy_pos+1):
            act_buy_pos = buy_pos - i
            if act_buy_pos == 0:
                #print("You do not have enough money to buy!")
                return
            total_amt = act_buy_pos*price*(1+self.handle_fee)
            if self.cash_balance > total_amt:
                self.position += act_buy_pos
                self.cash_balance -= total_amt
                self._mark_down_record(date,
                                       "Buy",
                                       price,
                                       act_buy_pos,
                                       -total_amt)
                print("Bought at",date,"with price =", price, "\tPos:", act_buy_pos)
                return
        
    
    def _sell(self,date,price):
        # Assume sell all position
        sell_pos = self.position
        total_amt = sell_pos*price*(1-self.handle_fee)
        if self.position >= 1:
            self.position -= sell_pos
            self.cash_balance += total_amt
            self._mark_down_record(date,
                                   "Sell",
                                   price,
                                   -sell_pos,
                                   total_amt)
            print("Sold at",date,"with price =", price, "\tPos:", sell_pos)
            
            return
        
    def _hold(self,date,price): # newly added
        self._mark_down_record(date,"Hold",price,0,0)

    def _run_do_nothing(self):
        balance = self.capital
        first_day_price = self.hpd.iloc[0,0]
        last_day_price = self.hpd.iloc[-1,0]
        buy_pos = floor(self.capital / first_day_price) # Calculate how many position can buy
        total_amt = buy_pos*first_day_price*(1+self.handle_fee) # Check if okay to buy (including fee)
        while self.capital < total_amt: # If not enough, reduce buy_pos by 1
            buy_pos -= 1
            total_amt = buy_pos*first_day_price*(1+self.handle_fee)
        position = buy_pos # Buy in 
        balance -= total_amt
        self.do_nth_profit = last_day_price*position+balance-self.capital

    def get_performance(self):
        if self._tested:
            model_profit = self.get_profit()
            do_nth_profit = self.get_do_nothing()
            perf = (model_profit-do_nth_profit)/abs(do_nth_profit)
            return perf
        else:
            print("No Backtesting Record.")

    def get_do_nothing(self):
        if self._tested:
            return self.do_nth_profit
        else:
            print("No Backtesting Record.")

    def get_profit(self):
        if self._tested:
            return self.get_cash_balance()+self.get_last_price()*self.get_position()-self.get_capital()
        else:
            print("No Backtesting Record.")

    def get_last_price(self):
        if self._tested:
            return self.last_price
        else:
            print("No Backtesting Record.")
    
    def get_position(self):
        if self._tested:
            return self.position
        else:
            print("No Backtesting Record.")
    
    def get_do_nothing_CAGR(self):
        if self._tested:
            start_bal = self.get_capital()
            end_bal = self.get_do_nothing()+self.get_capital()
            num_year = self.get_num_year()
            return pow(end_bal/start_bal,1/num_year)-1
        else:
            print("No Backtesting Record.")
            
    def get_model_CAGR(self):
        if self._tested:
            start_bal = self.get_capital()
            end_bal = self.get_profit()+self.get_capital()
            num_year = self.get_num_year()
            return pow(end_bal/start_bal,1/num_year)-1
        else:
            print("No Backtesting Record.")
            
    def get_diff_in_CAGR(self):
        if self._tested:
            return self.get_model_CAGR() - self.get_do_nothing_CAGR()
        else:
            print("No Backtesting Record.")
    
    def get_do_nothing_MDD(self):
        if self._tested:
            return (self._stock_trough-self._stock_peak)/self._stock_peak
        else:
            print("No Backtesting Record.")
            
    def get_model_MDD(self):
        if self._tested:
            return (self._portfolio_trough-self._portfolio_peak)/self._portfolio_peak
        else:
            print("No Backtesting Record.")
            
    def get_diff_in_MDD(self):
        if self._tested:
            return self.get_model_MDD() - self.get_do_nothing_MDD()
        else:
            print("No Backtesting Record.")
    
    def get_num_year(self):
        return self.num_year
    
    def get_cash_balance(self):
        return self.cash_balance
    
    def get_capital(self):
        return self.capital
    
    def get_amount(self):
        return self.get_capital()+self.get_profit()

    def print_do_nothing(self):
        if self._tested:
            print("If buy at", self.hpd.index[0],"with price =",self.hpd.iloc[0,0])
            print("and do nothing")
            print("Current Profit:",self.get_do_nothing())
        else:
            print("No Backtesting Record.")
            
    def print_performance(self):
        if self._tested:
            print("Performance:", str(round(self.get_performance()*100,2))+"%")
        else:
            print("No Backtesting Record.")
            
    def print_CAGR_performance(self):
        if self._tested:
            print("CAGR Performance:", str(round(self.get_diff_in_CAGR()*100,2))+"%")
        else:
            print("No Backtesting Record.")
    
    def print_trade_record(self):
        if self._tested:
            print(self.trade_record)
        else:
            print("No Backtesting Record.")
            print(self.trade_record)
    
    def print_profit(self):
        if self._tested:
            print("Current Profit with model:",self.get_profit())
        else:
            print("No Backtesting Record.")
    
    def export_trade_record(self,stock,add_msg = ""):
        if self._tested:
            # Save the trade record to the path
            if not add_msg == "":
                add_msg = "_" + add_msg
            self.trade_record = self.trade_record.sort_index()
            self.trade_record.to_csv(record_folder+stock+add_msg+".csv")
            print("Trade record exported.")
        else:
            print("No Backtesting Record.")

# Processing Function

In [ ]:
# Split the train and test data
def custom_split(data,start,end):
    train = (data.index >= start) & (data.index <= end)
    train_X = data[train]
    return train_X

In [ ]:
# Assume we use 5 days price data to predict closing price of the 6th day
num_day_to_predict = 5

In [ ]:
def produce_result_target_price(X,num_day,result_col_name = "result_price"):
    y = pd.DataFrame(np.nan, index=X.index, columns=[result_col_name])
    for i in range(num_day,len(X)):
        y.loc[X.index[i-1],result_col_name] = X.loc[X.index[i],"open"]
    return y

In [ ]:
def transform_x_data_to_one_vector(X,num_day):
    col_name = []
    for i in range(num_day):
        for j in X.columns:
            col_name.append(j+"-"+str(i))
    new_X = pd.DataFrame(np.nan, index=X.index, columns=col_name)
    
    for i in range(num_day-1,len(X)):
        for col in col_name:
            split_list = col.split("-")
            new_X.loc[X.index[i],col] = X.loc[X.index[i-int(split_list[1])],split_list[0]]
    
    return new_X

In [ ]:
def drop_nan_row_y(y,num_day):
    drop_list = [y.index[i] for i in range(num_day)]
    drop_list.append(y.index[-1])
    return y.drop(drop_list)

def drop_nan_row_X(X,num_day):
    drop_list = [X.index[i] for i in range(num_day)]
    drop_list.append(X.index[-1])
    return X.drop(drop_list)

In [ ]:
def get_label_for_svm(X,y):
    cols = []
    for col in X.columns:
        if "open-" in col:
            cols.append(col)
    X = X[cols]
    X["mean"] = X.mean(axis=1)
    y_ser = y.iloc[:,0]
    x_ser = X.loc[:,"mean"]
    compare = (y_ser > x_ser).to_list()
    action = list(map(lambda ele: 1 if ele else 0,compare)) # 1: Buy | 0: Sell
    return pd.Series(action, name = "Action")
    

In [ ]:
def normalize_data_by_row(X,y):
    norm_X = X.sub(X.mean(axis=0), axis=1).div(X.std(axis=0), axis=1)
    
    col_list =[] 
    for col in X.columns:
        if "open" in col:
            col_list.append(col)
    
    
    mean = np.mean(X.loc[:,col_list],axis=1)
    mean.index = y.index
    std = np.std(X.loc[:,col_list],axis=1)
    std.index = y.index
                         
    return (norm_X,y)

In [ ]:
def convert_prediction(test_X,pred):
    converted_pred = list(map(lambda p: "Buy" if p == 1 else "Sell",pred))
    return pd.DataFrame(converted_pred,index = test_X.index,columns = ["Action"])
    

# Load Model

In [ ]:
with open(model_folder+"svc_"+str(num_day_to_predict)+"day.pickle", "rb") as f:
    svc_test = pickle.load(f)

# Backtest Result for SVC

In [ ]:
import os

file_list = list(filter(lambda name: "SVC_" in name ,os.listdir(record_folder)))
if len(file_list) == 0:
  print("No such files...")
else:
  for f in file_list:
    print(file_list)
    os.remove(record_folder+f)
  print("All old files deleted...")

In [ ]:
#### SVC Backtest
backtest_summary = pd.DataFrame(columns=backtest_col)

for sector, stocks in stock_list.items():
    for stock in stocks:
        print("Runninng stock: "+ stock)
        stock_data_1 = web.DataReader(stock, "stooq",start=start_date, end=end_date)
        stock_data_1.columns = ["open","high","low","close","volume"]
        x_1 = StockDataFrame(stock_data_1)
        data_1 = x_1[['open','high','low','close','volume',
                      'boll', 'boll_ub', 'boll_lb',
                      'macd', 'macdh', 'macds',
                      'rsi_11', 'rsi_14', 'rsi_21']]
        data_1.index = [int(str(ind)[0:4]+str(ind)[5:7]+str(ind)[8:10]) for ind in data_1.index]
        data_1 = data_1.sort_index()
        print(stock + " Data loaded...")
        
        for k,v in date_range_list.items():
         
            print("Getting Data... - " + stock + " from " + str(k))
            if (data_1.index[0]>v):
                print("No data for this testing period... - " + stock + " from " + str(k))
                continue
            test_X = custom_split(data_1,start = k,end = v)
            if len(test_X) <= num_day_to_predict:
                print("No enoguh data for testing... - " + stock + " from " + str(k))
                continue
                
            print("Transforming Data... - " + stock + " from " + str(k))
            test_y = produce_result_target_price(test_X,num_day_to_predict)
            test_X_5 = transform_x_data_to_one_vector(test_X,num_day_to_predict)
            new_test_X_5  = drop_nan_row_X(test_X_5,num_day_to_predict)
            new_test_y = drop_nan_row_y(test_y,num_day_to_predict)
            new_action_test_y = get_label_for_svm(new_test_X_5,new_test_y)
            old_test_X_5 = new_test_X_5.copy()
            new_test_X_5,new_action_test_y = normalize_data_by_row(new_test_X_5,new_action_test_y)
            
            print("Predicting... - " + stock + " from " + str(k))
            svc_prediction = svc_test.predict(new_test_X_5)
            
            print("Converting... - " + stock + " from " + str(k))
            final_pred = convert_prediction(new_test_X_5,svc_prediction)
            final_test_y = new_test_y.rename(columns={"result_price":"Open"})
            final_pred = final_pred.sort_index()
            
            print("Backtesting... - " + stock + " from " + str(k))
            back1 = backtest(final_test_y,final_pred,10000,0.0005,period_year)
            back1.clear_trade_record(3)
            
            print("Start Iteration... - " + stock + " from " + str(k))
            back1.start_test()
            
            print("======= from "+ str(k) + " to " + str(v)+" =========")
            back1.print_do_nothing()
            back1.print_profit()
            back1.print_CAGR_performance()
            back1.export_trade_record("["+str(num_day_to_predict)+"]SVC_"+sector+"-"+stock,str(k)+"-to-"+str(v)+"("+str(round(back1.get_diff_in_CAGR()*100,2))+"%)")
            backtest_summary = backtest_summary.append({"Sector":sector,
                                                        "Stock":stock,
                                                        "Range_Start":str(k),
                                                        "Range_End":str(v),
                                                        do_nth_name+"_Profit":round(back1.get_do_nothing(),2),
                                                        model_name+"_Profit":round(back1.get_profit(),2),
                                                        do_nth_name+"_CAGR(in %)":round(back1.get_do_nothing_CAGR()*100,2),
                                                        model_name+"_CAGR(in %)":round(back1.get_model_CAGR()*100,2),
                                                        "CAGR_Performance(in %)":round(back1.get_diff_in_CAGR()*100,2),
                                                        do_nth_name+"_MDD(in %)":round(back1.get_do_nothing_MDD()*100,2),
                                                        model_name+"_MDD(in %)":round(back1.get_model_MDD()*100,2),
                                                        "MDD_Performance(in %)":round(back1.get_diff_in_MDD()*100,2)}, ignore_index=True)
            print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

print("Backtesting ["+str(num_day_to_predict)+"] Completed...")
backtest_summary.to_csv(backtest_summary_folder+"["+str(num_day_to_predict)+"]backtest_summary_SVC.csv")
print("Backtesting Summary Exported...")

In [ ]:
#### SVC
sd_backtest_summary = backtest_summary.groupby(["Sector","Stock"])[["Sector","Stock",
                                                          do_nth_name+"_Profit",
                                                          model_name+"_Profit",
                                                          do_nth_name+"_CAGR(in %)",
                                                          model_name+"_CAGR(in %)",
                                                          "CAGR_Performance(in %)",
                                                          do_nth_name+"_MDD(in %)",
                                                          model_name+"_MDD(in %)",
                                                          "MDD_Performance(in %)"]].std()

sd_backtest_summary = sd_backtest_summary.add_prefix("sd_")

sd_backtest_summary = sd_backtest_summary.round(decimals = 2)
sd_backtest_summary.to_csv(backtest_summary_folder+"["+str(num_day_to_predict)+"]backtest_summary_SVC_sd.csv")

